# Setup

In [ ]:
from collections import defaultdict
import json
import ee
import pandas as pd
import geopandas as gpd
import geemap

In [ ]:
ee.Authenticate()
ee.Initialize()

# Constants

In [ ]:
with open('popcodes.json', 'r') as popfile:
    popcodes2 = json.loads(popfile.read())

scale = 100

YEARS = [2000, 2005, 2010, 2015, 2020]

# Data

In [ ]:
ba_ic = ee.ImageCollection('JRC/GHSL/P2023A/GHS_BUILT_S')
ghspop_ic = ee.ImageCollection("JRC/GHSL/P2023A/GHS_POP")

# Functions

In [ ]:
def doOneYear(ufc, upd, ba_i, pop_ic, year):
    global ba_total
    global pop_total
    ba_res = defaultdict(list)
    pop_res = defaultdict(list)
    print("")
    print(year)
    for i in range(len(upd)):
        ua = upd.iloc[i]
        ua_f = ufc.filter(ee.Filter.eq('city_ids', str(ua['city_ids'])))
        geom = ua_f.geometry()
        if ua['country'] in popcodes2:
            local_ba = ba_i.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('built_surface').getInfo()
            localpop_img = pop_ic.select('population_count').first()
            localpop = localpop_img.reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e12).get('population_count').getInfo()
            ba_res[ua['country']].append(local_ba)
            pop_res[ua['country']].append(localpop)
            ba_total[year].append(local_ba)
            pop_total[year].append(localpop)
        print(i, end=' ')
    return ba_res, pop_res

# Caluculations

In [ ]:
urbext = {}
urbext_data = {}
bares = {}
popres = {}
ba_total = {}
pop_total = {}
for year in YEARS:
    urbext[year] = ee.FeatureCollection('projects/wri-datalab/SCL-Cities/urbanextents__bycountry_{0}_v7'.format(year))
    urbext_data[year] = geemap.ee_to_df(urbext[year])

for year in YEARS:
    ba_total[year] = []
    pop_total[year] = []
    bares[year], popres[year] = doOneYear(urbext[year], urbext_data[year], ba_ic.filter(ee.Filter.eq('system:index', str(year))).first(), ghspop_ic.filter(ee.Filter.calendarRange(year, year, 'year')), year)

# Sum & Format

In [ ]:
YEARPAIRS = [(2000, 2005), (2005, 2010), (2010, 2015), (2015, 2020)]
bycountry = {}
for idx, yearpair in enumerate(YEARPAIRS):
    year1, year2 = yearpair
    print('\n\n{0} {1}\n'.format(year1, year2))
    if idx == 0:
        bycountry['global'] = {
                'urban_built_area_{0}'.format(year1): sum(ba_total[year1]),
                'urban_population_{0}'.format(year1): sum(pop_total[year1]),
                'urban_built_area_{0}'.format(year2): sum(ba_total[year2]),
                'urban_population_{0}'.format(year2): sum(pop_total[year2]),
                'changerate_landconsumptionperperson_{0}-{1}'.format(year1, year2): (math.log(sum(ba_total[year2])/sum(pop_total[year2])) - math.log(sum(ba_total[year1])/sum(pop_total[year1]))) / (year2 - year1)
            }
        

        country_list = [i for i in list(set(bares[year2].keys())) if type(i)==str]
        country_list.sort()
        for country in country_list:
            bycountry[country] = {
                    'urban_built_area_{0}'.format(year1): sum(bares[year1][country]),
                    'urban_population_{0}'.format(year1): sum(popres[year1][country]),
                    'urban_built_area_{0}'.format(year2): sum(bares[year2][country]),
                    'urban_population_{0}'.format(year2): sum(popres[year2][country]),
                    'changerate_landconsumptionperperson_{0}-{1}'.format(year1, year2): (math.log(sum(bares[year2][country])/sum(popres[year2][country])) - math.log(sum(bares[year1][country])/sum(popres[year1][country]))) / (year2 - year1)
                }
    else:
        bycountry['global'] = bycountry['global'] | {
                'urban_built_area_{0}'.format(year2): sum(ba_total[year2]),
                'urban_population_{0}'.format(year2): sum(pop_total[year2]),
                'changerate_landconsumptionperperson_{0}-{1}'.format(year1, year2): (math.log(sum(ba_total[year2])/sum(pop_total[year2])) - math.log(sum(ba_total[year1])/sum(pop_total[year1]))) / (year2 - year1)
            }
        

        country_list = [i for i in list(set(bares[year2].keys())) if type(i)==str]
        country_list.sort()
        for country in country_list:
            bycountry[country] = bycountry[country] | {
                    'urban_built_area_{0}'.format(year2): sum(bares[year2][country]),
                    'urban_population_{0}'.format(year2): sum(popres[year2][country]),
                    'changerate_landconsumptionperperson_{0}-{1}'.format(year1, year2): (math.log(sum(bares[year2][country])/sum(popres[year2][country])) - math.log(sum(bares[year1][country])/sum(popres[year1][country]))) / (year2 - year1)
                }

# Output

In [ ]:
res_pd = pd.DataFrame.from_dict(bycountry, orient='index')
res_pd.index.name = 'geo'
res_pd.to_csv('CTY-5_landconsumption_changerate_v7.csv')